In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from sklearn import preprocessing

%matplotlib inline

2024-02-27 13:14:42.529389: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-27 13:14:42.553078: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-27 13:14:42.553102: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-27 13:14:42.553119: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-27 13:14:42.557477: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-27 13:14:42.558086: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [3]:
attacks = pd.read_csv("/home/knel/virtual_envs/ankh-morpork/ICS_data_generation/data/swat_processed_sample.csv", sep=",", usecols=range(1,23), skiprows=[0])

In [4]:
attacks = preprocessing.normalize(attacks.to_numpy(), norm="max", axis=0)

# swap sport to front
attacks[:, [0, 1]] = attacks[:, [1, 0]]
# swap dport to after sport
attacks[:, [1, 2]] = attacks[:, [2, 1]]
# swap protocols to after dport
attacks[:, [2, 14]] = attacks[:, [14, 2]]

# column order now -> sport, dport, protocols, continuous (discrete, discrete, discrete, continuous)



# should probably add batch and shuffle

train_dataset = attacks[:int(np.floor(attacks.shape[0]*3/4))]
test_dataset = attacks[int(np.floor(attacks.shape[0]*3/4)):]

num_features = attacks[:int(np.floor(attacks.shape[0]*3/4))].shape[1]
seq_length = 5

In [ ]:
len(train_dataset), len(test_dataset)

(1500, 500)

In [ ]:
train_dataset[2]

array([7.27481853e-01, 7.99678830e-01, 2.50000000e-01, 1.33689840e-03,
       2.29095074e-03, 1.30282912e-03, 1.94383728e-03, 5.15252969e-04,
       1.34779820e-04, 5.58035706e-04, 1.02306549e-03, 1.52555301e-03,
       7.65110941e-04, 4.49197871e-01, 2.15040043e-03, 2.15040043e-03,
       0.00000000e+00, 2.15040043e-03, 2.15040043e-03, 2.15040043e-03,
       0.00000000e+00, 2.15040043e-03])

In [ ]:
train_dataset[3]

array([5.53460684e-01, 6.60183781e-04, 2.50000000e-01, 2.67379679e-03,
       1.90912562e-03, 5.24761820e-03, 5.48529191e-03, 5.84350684e-05,
       1.01134633e-05, 3.20936160e-05, 2.22186570e-05, 2.28832952e-03,
       3.06044376e-03, 7.43833008e-01, 8.10130162e-02, 8.10130162e-02,
       0.00000000e+00, 8.10130162e-02, 8.10130162e-02, 8.10130162e-02,
       0.00000000e+00, 8.10130162e-02])

In [ ]:
# custom loss
def wasserstein_loss(y_true, y_pred):
    return tf.keras.backend.mean(y_true * y_pred)

In [ ]:
# clip model weights to a given hypercube
class ClipConstraint(tf.keras.constraints.Constraint):
	# set clip value when initialized
	def __init__(self, clip_value):
		self.clip_value = clip_value

	# clip model weights to hypercube
	def __call__(self, weights):
		return tf.keras.backend.clip(weights, -self.clip_value, self.clip_value)

	# get the config
	def get_config(self):
		return {'clip_value': self.clip_value}

In [ ]:
# noise will have same dimension as one flow

def make_generator_model(input_dim):
    """
    The generator takes noise as input and goes through a couple
    of hidden layers. Then, it splits the output into 3 softmax layers
    (one for each categorical variable with the same dimension as the
    number of possible values that variable can take) and a dense
    layer for all the rest of the continuous variables.

    Still experimenting with whether to compile the model here or not.
    """
    input = tf.keras.layers.Input(shape=input_dim, name="generator input")
    hidden = tf.keras.layers.Dense(80, activation="relu")(input)
    hidden = tf.keras.layers.Dense(80, activation="relu")(hidden)

    sport_hidden = tf.keras.layers.Dense(23030, name="sport_hidden")(hidden)
    dport_hidden = tf.keras.layers.Dense(14372, name="dport_hidden")(hidden)
    proto_hidden = tf.keras.layers.Dense(7, name="proto_hidden")(hidden)

    # dense output for continuous, softmax for categorical
    sport_output = tf.keras.layers.Softmax(1, name="sport_output")(sport_hidden)
    dport_output = tf.keras.layers.Softmax(1, name="dport_output")(dport_hidden)
    proto_output = tf.keras.layers.Softmax(1, name="proto_output")(proto_hidden)
    cont_output = tf.keras.layers.Dense(num_features-3, name="cont_output")(hidden)

    model = tf.keras.Model(inputs=input, outputs=[sport_output, dport_output, proto_output, cont_output],
                           name="Generator")
    opt = tf.keras.optimizers.RMSprop(learning_rate=0.00005)
    # model.compile(loss=wasserstein_loss, optimizer=opt)

    return model

In [ ]:
def make_simple_generator(input_dim):
    model = tf.keras.Sequential(name="SimpleGenerator")
    model.add(tf.keras.layers.InputLayer(input_shape=(input_dim,)))
    model.add(tf.keras.layers.Dense(128, activation="relu"))
    model.add(tf.keras.layers.Dense(256, activation="relu"))
    model.add(tf.keras.layers.Dense(512, activation="relu"))
    model.add(tf.keras.layers.Dense(256, activation="relu"))
    model.add(tf.keras.layers.Dense(128, activation="relu"))

    # Output layer
    model.add(tf.keras.layers.Dense(num_features, activation="linear"))

    return model


In [ ]:
def convert_generator_output(arr):
    """
    Convert the output of the generator into the format expected for training.
    This is so the generated data has the same shape as the real data.
    """
    num_samples = arr.shape[0]
    final = np.zeros((num_samples, 22))  # Assuming 22 is the desired output size

    # Split the generator output into sport, dport, proto, and cont
    sport_dist = arr[:, :23030]
    dport_dist = arr[:, 23030:23030 + 14372]
    proto_dist = arr[:, 23030 + 14372:23030 + 14372 + 7]
    cont = arr[:, 23030 + 14372 + 7:]

    # Assign the values to the final array
    final[:, :23030] = sport_dist
    final[:, 23030:23030 + 14372] = dport_dist
    final[:, 23030 + 14372:23030 + 14372 + 7] = proto_dist
    final[:, 23030 + 14372 + 7:] = cont

    return final

In [ ]:
def convert_generator_output(arr):
    """
    For each categorical feature, takes the most likely probability given
    by the Softmax output and concatenates it with the continuous features.
    This is so the generated data has the same shape as the real data.
    """
    sport_dist, dport_dist, proto_dist, cont = arr
    num_samples = sport_dist.shape[0]
    final = np.zeros(22)
    for sample_index in range(num_samples):
        sport = np.array([np.argmax(sport_dist[sample_index])+1])
        dport = np.array([np.argmax(dport_dist[sample_index])+1])
        proto = np.array([np.argmax(proto_dist[sample_index])+1])
        data_point = np.concatenate((sport, dport, proto, cont[sample_index]))
        final = np.vstack((final, data_point))
    return tf.convert_to_tensor(final[1:])

In [ ]:
def make_critic_model(input_shape):
    """
    Critic (discriminator) more or less as described by the WGAN people.
    The output layer uses linear activation since this is a critic and not
    a standard discriminator.
    """
    const = ClipConstraint(0.01)

    input = tf.keras.layers.Input(shape=input_shape, name="discriminator input")
    hidden = tf.keras.layers.LSTM(100, recurrent_dropout=0.4, return_sequences=True, kernel_constraint=const)(input)
    hidden = tf.keras.layers.BatchNormalization()(hidden)
    hidden = tf.keras.layers.LSTM(100, recurrent_dropout=0.4, kernel_constraint=const)(hidden)
    hidden = tf.keras.layers.BatchNormalization()(hidden)
    output = tf.keras.layers.Dense(1, activation="linear")(hidden)

    model = tf.keras.Model(inputs=input, outputs=output, name="Critic")

    # opt = tf.keras.optimizers.RMSprop(learning_rate=0.00005)
    # model.compile(loss=wasserstein_loss, optimizer=opt)

    return model

In [ ]:
def make_gan(generator, critic, gen_input_dim):
    """
    Would be a convenient way to train the GAN but this doesn't work
    because the generator output has 4 tensors (one for each layer)
    and the critic expects an input with the same dimension as the real data.

    We can't just change the shape of the real data because I'd have to one-hot encode
    the categorical variables - these datasets are huge and it would take FOREVER.
    """
    for layer in critic.layers:
        if not isinstance(layer, tf.keras.layers.BatchNormalization):
            layer.trainable = False

    input = tf.keras.layers.Input(shape=gen_input_dim, name="combined input")
    x = generator(input)
    x = critic(x)
    model = tf.keras.Model(inputs=input, outputs=x)
    opt = tf.keras.optimizers.legacy.RMSprop(learning_rate=0.00005)
    model.compile(loss=wasserstein_loss, optimizer=opt)
    return model

In [ ]:
# Some helper functions that may or may not be useful

def generate_latent_points(latent_dim, n_samples):
	"""
	Generate points in latent space as input for the generator.
	"""
	# generate points in the latent space
	x_input = np.random.randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input

def generate_real_samples(dataset, n_samples):
	"""
	Select real samples.
	"""
	# choose random instances
	ix = np.random.randint(0, dataset.shape[0], n_samples)
	# select data
	X = dataset[ix]
	# refactor real data to have same shape as generated data (4 tensors)

	# generate class labels, -1 for 'real'
	y = -np.ones((n_samples, 1))
	return X, y

def generate_fake_samples(generator, latent_dim, n_samples):
	"""
	Use the generator to generate n fake examples, with class labels
	"""

	# generate points in latent space
	x_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	X = generator.predict(x_input)
	X = convert_generator_output(X)
	# create class labels with 1.0 for 'fake'
	y = np.ones((n_samples, 1))
	return X, y

In [ ]:
# Wasserstein loss for critic
def critic_loss(pred_real, pred_fake):
    return tf.keras.backend.mean(pred_real * pred_fake)

# Wasserstein loss for generator
def generator_loss(pred_fake):
    return -tf.keras.backend.mean(pred_fake)

In [ ]:
generator_optimiser = tf.keras.optimizers.legacy.RMSprop(learning_rate=0.00005)
critic_optimiser = tf.keras.optimizers.legacy.RMSprop(learning_rate=0.00005)
generator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

In [ ]:
def train_step(generator, critic, dataset, latent_dim, n_batch=64, n_critic=5):

    for _ in range(n_critic):

        """
        Train critic more often than the generator for WGAN
        """

        with tf.GradientTape() as critic_tape:
            # Critic training works fine
            X_real, y_real = generate_real_samples(dataset, n_batch)
            noise = generate_latent_points(latent_dim, n_batch)
            X_fake = generator(noise, training=False)
            X_fake = convert_generator_output(X_fake)

            pred_real = critic(X_real, training=True)
            pred_fake = critic(X_fake, training=True)

            c_loss = critic_loss(pred_real, pred_fake)

        critic_gradients = critic_tape.gradient(c_loss, critic.trainable_variables)
        critic_optimiser.apply_gradients(zip(critic_gradients, critic.trainable_variables))
    print("In here after discriminator")
    """
    Train generator
    """

    with tf.GradientTape() as gen_tape:
        X_fake = generator(noise, training=True)
        X_fake = convert_generator_output(X_fake)
        pred_fake = critic(X_fake, training=False)
        g_loss = generator_loss(pred_fake)
        print(gen_tape.gradient)
    tf.print("Generator Loss:", g_loss)
    print("Generator Trainable Variables:", generator.trainable_variables)
    asdasd
    # These gradients are all None and I can't figure out why
    # Are the multiple output layers of the generator a problem here?
    # Maybe the definition of the loss function?
    generator_gradients = gen_tape.gradient(g_loss, generator.trainable_variables)
    print("Critic Gradients:", critic_gradients)
    print("Generator Gradients:", generator_gradients)
    easdasd

    generator_optimiser.apply_gradients(zip(generator_gradients, generator.trainable_variables))



    return c_loss, g_loss

In [ ]:
def train_step(generator, critic, dataset, latent_dim, n_batch=64, n_critic=5):
    # 1. Train critic
    for _ in range(n_critic):
        with tf.GradientTape() as critic_tape:
            # Generate real and fake samples
            X_real, y_real = generate_real_samples(dataset, n_batch)
            noise = generate_latent_points(latent_dim, n_batch)
            X_fake = generator(noise, training=False)
            X_fake = convert_generator_output(X_fake)

            # Compute critic predictions for real and fake samples
            pred_real = critic(X_real, training=True)
            pred_fake = critic(X_fake, training=True)

            # Compute critic loss
            c_loss = critic_loss(pred_real, pred_fake)

        # Update critic weights
        critic_gradients = critic_tape.gradient(c_loss, critic.trainable_variables)
        critic_optimiser.apply_gradients(zip(critic_gradients, critic.trainable_variables))

    # Generate noise for the generator training
    noise = generate_latent_points(latent_dim, n_batch)

    # 2. Train generator
    with tf.GradientTape() as gen_tape:
        # Generate fake samples for the current batch
        noise = generate_latent_points(latent_dim, n_batch)
        X_fake = generator(noise, training=True)
        X_fake = convert_generator_output(X_fake)

        # Compute critic predictions for the generated samples
        pred_fake = critic(X_fake, training=False)

        # Compute generator loss
        g_loss = generator_loss(pred_fake)


    print("Generated Output:", X_fake)
    tf.print("Generator Loss:", g_loss)
    print("Generator summary ", generator.summary())
    print("Trainable variables: ", generator.trainable_variables)
    # Compute generator gradients
    generator_gradients = gen_tape.gradient(g_loss, generator.trainable_variables)
    print("NaN in Generator Output:", tf.reduce_any(tf.math.is_nan(X_fake)))
    print("NaN in Generator Loss:", tf.reduce_any(tf.math.is_nan(g_loss)))
    print("Min Generator Output:", tf.reduce_min(X_fake))
    print("Max Generator Output:", tf.reduce_max(X_fake))
    print("Inf in Generator Output:", tf.reduce_any(tf.math.is_inf(X_fake)))
    print("Inf in Generator Loss:", tf.reduce_any(tf.math.is_inf(g_loss)))
    for grad, var in zip(generator_gradients, generator.trainable_variables):
      print(f"Gradient for {var.name}:", grad)
    print("Gen gradients : ", generator_gradients)
    # Update generator weights
    generator_optimiser.apply_gradients(zip(generator_gradients, generator.trainable_variables))

    # Print and return losses
    tf.print("Critic Loss:", c_loss)
    tf.print("Generator Loss:", g_loss)

    return c_loss, g_loss

In [ ]:
def train(dataset, n_epochs=2, latent_dim=50):

    generator = make_generator_model(latent_dim)
    generator = make_simple_generator(latent_dim)
    critic = make_critic_model((num_features, 1))
    generator.compile(optimizer=generator_optimiser, loss=generator_loss)
    c_losses, g_losses = [], []

    for _ in range(n_epochs):
        for batch in dataset:
            c_loss, g_loss = train_step(generator, critic, dataset, latent_dim)
            c_losses.append(c_loss)
            g_losses.append(g_loss)

    return c_losses, g_losses

In [ ]:
# Test training on a small sample of the dataset
dataset = attacks[:10]
c_losses, g_losses = train(dataset, 2, 10)

Generated Output: [[ 0.03682016 -0.13725437  0.01530193 ...  0.00323969  0.00109265
   0.02656257]
 [ 0.12080162 -0.12529905  0.07918794 ...  0.02806921  0.03629631
   0.00331291]
 [ 0.13487729 -0.18661509  0.02867277 ... -0.03999968 -0.11089537
  -0.00321516]
 ...
 [ 0.01401116 -0.07144432  0.03908198 ...  0.0269659   0.00047158
  -0.00950964]
 [ 0.06002793 -0.13166794  0.06363656 ...  0.02195042  0.02467676
  -0.05509738]
 [ 0.14546759 -0.14995448  0.0473679  ...  0.04530958  0.0933165
  -0.03924917]]
Generator Loss: -0.000300596264
Model: "SimpleGenerator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_50 (Dense)            (None, 128)               1408      
                                                                 
 dense_51 (Dense)            (None, 256)               33024     
                                                                 
 dense_52 (Dense)            (None, 51

ValueError: No gradients provided for any variable: (['dense_50/kernel:0', 'dense_50/bias:0', 'dense_51/kernel:0', 'dense_51/bias:0', 'dense_52/kernel:0', 'dense_52/bias:0', 'dense_53/kernel:0', 'dense_53/bias:0', 'dense_54/kernel:0', 'dense_54/bias:0', 'dense_55/kernel:0', 'dense_55/bias:0'],). Provided `grads_and_vars` is ((None, <tf.Variable 'dense_50/kernel:0' shape=(10, 128) dtype=float32, numpy=
array([[-0.16613153, -0.04200545,  0.15716307, ...,  0.15045272,
         0.14351399,  0.11724181],
       [-0.03295447,  0.2004156 , -0.00451599, ..., -0.1395693 ,
        -0.1571418 , -0.04516445],
       [ 0.1617022 , -0.06330299, -0.10965457, ..., -0.01647766,
         0.04076628, -0.14750217],
       ...,
       [ 0.10820521,  0.03061028, -0.01889439, ...,  0.14297156,
        -0.1828673 , -0.19050944],
       [-0.149292  ,  0.05717115, -0.09174094, ..., -0.04277968,
        -0.10745165,  0.11147936],
       [ 0.0389486 ,  0.16030927, -0.13768798, ...,  0.15474252,
         0.1746719

In [ ]:
# generator = make_generator_model(50)
# critic = make_critic_model((num_features, 1))
# noise = generate_latent_points(50, 3)
# X_fake = generator(noise, training=False)
# X_fake = convert_generator_output(X_fake)
# y_fake = critic(X_fake)
# g_loss = generator_loss(y_fake)

# generator.compile(loss=generator_loss, optimizer=generator_optimiser)
# gl = generator(noise)
# gl = convert_generator_output(gl)
# yg = critic(gl)
# generator.train_on_batch(noise, yg)

# g_loss, yg

In [ ]:
# # size of the latent space
# latent_dim = 50
# # create the critic
# critic = make_critic_model((num_features, 1))
# # create the generator
# generator = make_generator_model(latent_dim)
# # create the gan
# gan_model = make_gan(generator, critic, latent_dim)
# # load image data

# epochs = 10
# # train model
# critic_loss_real, critic_loss_fake, generator_loss = train(generator, critic, dataset, latent_dim, n_epochs=epochs)

In [ ]:
# plt.plot(critic_loss_fake, label="crit")
# plt.plot(generator_loss, label="gen")
# plt.legend()
# plt.show()

In [ ]:
# # train the generator and critic
# def train(g_model, c_model, gan_model, dataset, latent_dim, n_epochs=2, n_batch=64, n_critic=5):
# 	# calculate the number of batches per training epoch
# 	bat_per_epo = int(dataset.shape[0] / n_batch)
# 	# calculate the number of training iterations
# 	n_steps = bat_per_epo * n_epochs
# 	# calculate the size of half a batch of samples
# 	half_batch = int(n_batch / 2)
# 	# lists for keeping track of loss
# 	c1_hist, c2_hist, g_hist = list(), list(), list()
# 	# manually enumerate epochs
# 	for i in range(n_steps):
# 		# update the critic more than the generator
# 		c1_tmp, c2_tmp = list(), list()
# 		for _ in range(n_critic):
# 			# get randomly selected 'real' samples
# 			X_real, y_real = generate_real_samples(dataset, half_batch)
# 			# update critic model weights
# 			c_loss1 = c_model.train_on_batch(X_real, y_real)
# 			c1_tmp.append(c_loss1)
# 			# generate 'fake' examples
# 			X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
# 			# update critic model weights
# 			c_loss2 = c_model.train_on_batch(X_fake, y_fake)
# 			c2_tmp.append(c_loss2)
# 		# store critic loss
# 		c1_hist.append(np.mean(c1_tmp))
# 		c2_hist.append(np.mean(c2_tmp))
# 		# prepare points in latent space as input for the generator
# 		X_gan = generate_latent_points(latent_dim, n_batch)
# 		print(X_gan.shape)
# 		# create inverted labels for the fake samples
# 		y_gan = -np.ones((n_batch, 1))
# 		# update the generator via the critic's error
# 		g_loss = gan_model.train_on_batch(X_gan, y_gan)
# 		g_hist.append(g_loss)
# 		# summarize loss on this batch
# 		print('>%d, c1=%.3f, c2=%.3f g=%.3f' % (i+1, c1_hist[-1], c2_hist[-1], g_loss))
# 		plot_history(c1_hist, c2_hist, g_hist)

In [7]:
import torch
from torch.utils.data import Dataset, DataLoader

class Dataset(Dataset):
    def __init__(self, _data, num_classes_list):
        self.data = torch.tensor(_data, dtype=torch.float32)
        self.num_classes_list = num_classes_list

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]

        # One-hot encode each of the first three values with different num_classes
        one_hot_encoded_1 = torch.nn.functional.one_hot(sample[0].long(), self.num_classes_list[0])
        one_hot_encoded_2 = torch.nn.functional.one_hot(sample[1].long(), self.num_classes_list[1])
        one_hot_encoded_3 = torch.nn.functional.one_hot(sample[2].long(), self.num_classes_list[2])
        # print("1 :",one_hot_encoded_1.shape)
        # print("2 :",one_hot_encoded_2.shape)
        # print("3 :",one_hot_encoded_3.shape)
        # print("4 :",sample[3:].shape)
        # Concatenate one-hot encoding with the remaining values
        modified_sample = torch.cat((one_hot_encoded_1.float(), one_hot_encoded_2.float(), one_hot_encoded_3.float(), sample[3:]))

        return modified_sample

data = np.array(train_dataset[:10000])
dataset = Dataset(data,[23030, 14372, 7])

batch_size = 64
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4, drop_last=True)


In [8]:
len(dataset[1])

37428

In [9]:
def differentiable_argmax(gumbel_softmax_sample):
    _, max_indices = gumbel_softmax_sample.max(dim=-1, keepdim=True)
    one_hot = torch.zeros_like(gumbel_softmax_sample).scatter_(-1, max_indices, 1.0)
    return one_hot - gumbel_softmax_sample.detach() + gumbel_softmax_sample

In [10]:
def convert_generator_output(output_tensors):
    sport_dist, dport_dist, proto_dist, cont = output_tensors
    num_samples = sport_dist.size(0)
    final = torch.zeros((num_samples, 22))

    for sample_index in range(num_samples):
        sport = torch.argmax(sport_dist[sample_index]) + 1
        dport = torch.argmax(dport_dist[sample_index]) + 1
        proto = torch.argmax(proto_dist[sample_index]) + 1

        data_point = torch.cat((sport.view(1), dport.view(1), proto.view(1), cont[sample_index]))
        final[sample_index] = data_point

    return final

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Gumbel
num_features=22
# Generator model

class Generator(nn.Module):
    def __init__(self, input_dim, num_features):
        super(Generator, self).__init__()

        self.hidden1 = nn.Linear(input_dim, 80)
        self.hidden2 = nn.Linear(80, 80)

        self.sport_hidden = nn.Linear(10, 23030)
        self.dport_hidden = nn.Linear(10, 14372)
        self.proto_hidden = nn.Linear(10, 7)

        # self.sport_output = nn.Softmax(dim=1)
        # self.dport_output = nn.Softmax(dim=1)
        # self.proto_output = nn.Softmax(dim=1)

        self.cont_output = nn.Linear(50, num_features - 3)

    def forward(self, x):
        x = torch.relu(self.hidden1(x))
        x = torch.relu(self.hidden2(x))
        # print(x.shape)
        _sport_hidden = self.sport_hidden(x[:,:10])
        _dport_hidden = self.dport_hidden(x[:,10:20])

        _proto_hidden = self.proto_hidden(x[:,20:30])
        # sport_output = self.sport_output(sport_hidden)
        # dport_output = self.dport_output(dport_hidden)
        # proto_output = self.proto_output(proto_hidden)

        # Do not apply torch.argmax here, let the loss function handle it
        cont_output = self.cont_output(x[:,30:])

        # Concatenate the tensors along dimension 1
        output_tensor = torch.cat((_sport_hidden, _dport_hidden, _proto_hidden, cont_output), dim=1)
        return output_tensor
        # return sport_output, dport_output, proto_output, cont_output
# Discriminator model
class Discriminator(nn.Module):
    def __init__(self, input_size, hidden_size=100, recurrent_dropout=0.4):
        super(Discriminator, self).__init__()

        # LSTM layers
        self.lstm1 = nn.LSTM(input_size, hidden_size, batch_first=True, bidirectional=True, dropout=recurrent_dropout)
        self.lstm2 = nn.LSTM(hidden_size * 2, hidden_size, batch_first=True, bidirectional=True, dropout=recurrent_dropout)

        # Batch normalization layers
        self.batch_norm1 = nn.BatchNorm1d(hidden_size * 2)
        self.batch_norm2 = nn.BatchNorm1d(hidden_size * 2)

        # Linear output layer
        self.output_layer = nn.Linear(hidden_size * 2, 1)

    def forward(self, x):
        # LSTM layers
        out, _ = self.lstm1(x)
        # print(out.shape)
        out = self.batch_norm1(out)
        # print(out.shape)
        out, _ = self.lstm2(out)
        # print(out.shape)
        out = self.batch_norm2(out)
        # print(out.shape)

        # Global average pooling
        # out = torch.mean(out, dim=1)
        # print(out.shape)
        # Output layer
        # out = self.output_layer(out)
        out = torch.sigmoid(self.output_layer(out))


        return out  # Fix for the dimension mismatch

# Hyperparameters
latent_dim = 10
data_dim = len(dataset[1])
lr = 0.0002
batch_size = 64
epochs = 10

# Models
generator = Generator(latent_dim, num_features)
discriminator = Discriminator(data_dim)
max_grad_norm = 1.0
# Loss function and optimizers
criterion = nn.BCELoss()
criterion_gen = nn.SmoothL1Loss()
# criterion = nn.CrossEntropyLoss()
optimizer_g = optim.Adam(generator.parameters(), lr=lr)
optimizer_d = optim.Adam(discriminator.parameters(), lr=lr)

# Training loop
import csv
import copy
from tqdm import tqdm
parameters_list = []
csv_file_path = "generator_parameters.csv"
# with open(csv_file_path, mode='w', newline='') as csv_file:
#     fieldnames = ["Epoch", "Parameter", "Value"]
#     csv_writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

#     # Write the header
#     csv_writer.writeheader()

generator.train()
discriminator.train()
for epoch in tqdm(range(epochs)):
    total_loss_fake = 0
    total_loss_real = 0
    total_loss_gen = 0
    for _ in data_loader:
        # Train Discriminator
        real_data = _
        real_labels = torch.ones((batch_size, 1))
        fake_labels = torch.zeros((batch_size, 1))

        optimizer_d.zero_grad()

        # Real data
        # print("Real data: ", real_data.shape)
        output_real = discriminator(real_data)
        # print(output_real)
        loss_real = criterion(output_real, real_labels)
        # print("Loss:", loss_real.item())
        loss_real.backward()
        # print("Gradients of a specific layer:", generator.hidden1.weight.grad)
        total_loss_real+=loss_real

        # Fake data
        noise = torch.randn(batch_size, latent_dim)
        output = generator(noise)
        # print(fake_data.shape)
        sport_output = F.softmax(output[:, :23030], dim=1)
        dport_output = F.softmax(output[:, 23030:23030+14372], dim=1)
        proto_output = F.softmax(output[:, 23030+14372:23030+14372+7], dim=1)
        cont_output = output[:, 23030+14372+7:]
        # print(sport_output.shape)
        # print(cont_output.shape)
        fake_data = torch.cat((sport_output, dport_output, proto_output, cont_output), dim=1)
        # fake_data = torch.cat((output[:,1].unsqueeze(1),output[:,2].unsqueeze(1),output[:,3].unsqueeze(1),cont_output), dim=1)
        # print(fake_data.shape)

        # print("shape ", fake_data.shape)
        # print(fake_data)

        # print("Fake data :", len(_))
        # for array in _:
        #   print("Shape of array:", array.shape)
        # print(_[2])
        # fake_data = convert_generator_output(_)
        # print(fake_data.shape)
        # print(fake_data)

        output_fake = discriminator(fake_data)
        # print(output_fake)
        # print(fake_labels)
        loss_fake = criterion(output_fake, fake_labels)
        loss_fake.backward()
        total_loss_fake+=loss_fake
        # print("Gradients of a specific layer:", generator.hidden1.weight.grad)
        optimizer_d.step()

        # Train Generator
        optimizer_g.zero_grad()
        # generated_data = generator(noise)
        output = generator(noise)
        # print(fake_data.shape)
        sport_output = F.softmax(output[:, :23030], dim=1)
        dport_output = F.softmax(output[:, 23030:23030+14372], dim=1)
        proto_output = F.softmax(output[:, 23030+14372:23030+14372+7], dim=1)
        # sport_hidden = output[:, :23030]
        # dport_hidden = output[:, 23030:23030+14372]
        # proto_hidden = output[:, 23030+14372:23030+14372+7]
        cont_output = output[:, 23030+14372+7:]
        # temperature = 1.0  # You can adjust the temperature hyperparameter
        # gumbel_dist = Gumbel(0, 1).sample(sport_hidden.shape)
        # sport_output = F.softmax((sport_hidden + gumbel_dist) / temperature, dim=1)

        # gumbel_dist = Gumbel(0, 1).sample(dport_hidden.shape)
        # dport_output = F.softmax((dport_hidden + gumbel_dist) / temperature, dim=1)

        # gumbel_dist = Gumbel(0, 1).sample(proto_hidden.shape)
        # proto_output = F.softmax((proto_hidden + gumbel_dist) / temperature, dim=1)
        # print(proto_output.shape)
        # print(proto_output)
        # print(differentiable_argmax(proto_output))
        # print(differentiable_argmax(proto_output)[:,1])

        # print(cont_output.shape)

        # generated_data = torch.cat((differentiable_argmax(sport_output).unsqueeze(1),differentiable_argmax(dport_output)[:,1].unsqueeze(1),differentiable_argmax(proto_output)[:,1].unsqueeze(1), cont_output), dim=1)
        # generated_data = torch.cat((sport_output[:,1].unsqueeze(1),dport_output[:,2].unsqueeze(1),proto_output[:,3].unsqueeze(1),cont_output), dim=1)
        generated_data = torch.cat((sport_output, dport_output, proto_output, cont_output), dim=1)
        # print(generated_data.shape)


        # generated_data = convert_generator_output(generator(noise))
        # print(generated_data.shape)
        output_generated = discriminator(generated_data)
        # loss_gen = criterion(output_generated, real_labels)
        loss_gen = criterion_gen(output_generated, real_labels.float())
        # print("Discriminator Output:", output_generated)
        # print("Real Labels:", real_labels)
        # print("Loss:", loss_gen.item())
        loss_gen.backward()
        # torch.nn.utils.clip_grad_norm_([generator.sport_hidden.weight], max_grad_norm)
        total_loss_gen+=loss_gen
        # for param in generator.sport_hidden.parameters():
        #   print(param.requires_grad)
        # for name, param in generator.named_parameters():
        #   if name == 'sport_hidden.weight':
        #       print(name, param.requires_grad)
        #       print(name, param.grad)
        #       print(f"{name} - Gradient Statistics: Mean={param.grad.mean()}, Std={param.grad.std()}, Min={param.grad.min()}, Max={param.grad.max()}")

        # asdasdasd
        # print("Gradients of a specific layer:", generator.sport_hidden.weight.grad)
        optimizer_g.step()

    # Print losses at the end of each epoch
    print(f"Epoch [{epoch+1}/{epochs}], D Loss: {total_loss_real + total_loss_fake}, G Loss: {total_loss_gen}")
    print(f"\nEpoch {epoch + 1} - Parameters of Generator:\n")
    epoch_parameters = {"Epoch": epoch + 1}
    for name, param in generator.named_parameters():
        # print(name, param.data)
        # Store parameters in the dictionary
        # epoch_parameters[name] = param.data
        epoch_parameters[name] = copy.deepcopy(param.data)

    # Append epoch parameters to the dictionary
    parameters_list.append(epoch_parameters)




/home/knel/virtual_envs/ankh-morpork/lib/python3.11/site-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.4 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
 10%|█         | 1/10 [00:16<02:27, 16.35s/it]

Epoch [1/10], D Loss: 32.52910232543945, G Loss: 2.936279535293579

Epoch 1 - Parameters of Generator:



 20%|██        | 2/10 [00:31<02:06, 15.85s/it]

Epoch [2/10], D Loss: 32.10264587402344, G Loss: 2.907186508178711

Epoch 2 - Parameters of Generator:



 30%|███       | 3/10 [00:47<01:50, 15.72s/it]

...

Epoch [8/10], D Loss: 31.961471557617188, G Loss: 2.88663387298584

Epoch 8 - Parameters of Generator:



 90%|█████████ | 9/10 [02:20<00:15, 15.52s/it]

Epoch [9/10], D Loss: 31.966236114501953, G Loss: 2.886303424835205

Epoch 9 - Parameters of Generator:



100%|██████████| 10/10 [02:36<00:00, 15.60s/it]

Epoch [10/10], D Loss: 31.95956802368164, G Loss: 2.886603593826294

Epoch 10 - Parameters of Generator:



In [ ]:
csv_file_path = "generator_parameters_1.csv"
with open(csv_file_path, mode='w', newline='') as csv_file:
    fieldnames = ["Epoch"] + list(generator.state_dict().keys())
    csv_writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    print(fieldnames)
    # Write the header
    csv_writer.writeheader()

    # Write parameters for each epoch
    for i in parameters_list:
      # print(i)
      # for key in i.keys():
      # print(i.values())
      print(i.keys())
      row = {}
      for key in i.keys():
        row[key] = i[key]
      print(row.keys())
      csv_writer.writerow(i)

      # exit
      # csv_writer.writerows(i.values())

['Epoch', 'hidden1.weight', 'hidden1.bias', 'hidden2.weight', 'hidden2.bias', 'sport_hidden.weight', 'sport_hidden.bias', 'dport_hidden.weight', 'dport_hidden.bias', 'proto_hidden.weight', 'proto_hidden.bias', 'cont_output.weight', 'cont_output.bias']
dict_keys(['Epoch', 'hidden1.weight', 'hidden1.bias', 'hidden2.weight', 'hidden2.bias', 'sport_hidden.weight', 'sport_hidden.bias', 'dport_hidden.weight', 'dport_hidden.bias', 'proto_hidden.weight', 'proto_hidden.bias', 'cont_output.weight', 'cont_output.bias'])
dict_keys(['Epoch', 'hidden1.weight', 'hidden1.bias', 'hidden2.weight', 'hidden2.bias', 'sport_hidden.weight', 'sport_hidden.bias', 'dport_hidden.weight', 'dport_hidden.bias', 'proto_hidden.weight', 'proto_hidden.bias', 'cont_output.weight', 'cont_output.bias'])
dict_keys(['Epoch', 'hidden1.weight', 'hidden1.bias', 'hidden2.weight', 'hidden2.bias', 'sport_hidden.weight', 'sport_hidden.bias', 'dport_hidden.weight', 'dport_hidden.bias', 'proto_hidden.weight', 'proto_hidden.bias', 'c

In [ ]:
print(parameters_list[1]['cont_output.bias'])



tensor([-0.0939,  0.0259,  0.0789, -0.1096, -0.0247,  0.1134,  0.0200, -0.0971,
         0.0724, -0.0288,  0.1037, -0.0800,  0.0062,  0.0384,  0.0834, -0.0506,
         0.0043, -0.1062, -0.0500])


In [ ]:
print(parameters_list[0]['cont_output.bias'])

tensor([-0.0943,  0.0255,  0.0768, -0.1122, -0.0260,  0.1110,  0.0208, -0.0991,
         0.0731, -0.0295,  0.1050, -0.0800,  0.0076,  0.0349,  0.0833, -0.0497,
         0.0037, -0.1050, -0.0500])


In [ ]:
}import tensorflow as tf
import numpy as np

# Assuming num_features is defined somewhere
num_features = 30

# Assuming wasserstein_loss is defined somewhere
def wasserstein_loss(y_true, y_pred):
    # Placeholder for wasserstein_loss function
    return tf.reduce_mean(y_true * y_pred)

def make_generator_model(input_dim):
    input = tf.keras.layers.Input(shape=input_dim, name="generator input")
    hidden = tf.keras.layers.Dense(80, activation="relu")(input)
    hidden = tf.keras.layers.Dense(80, activation="relu")(hidden)
    print(hidden.shape)
    sport_hidden = tf.keras.layers.Dense(23030, name="sport_hidden")(hidden)
    dport_hidden = tf.keras.layers.Dense(14372, name="dport_hidden")(hidden)
    proto_hidden = tf.keras.layers.Dense(7, name="proto_hidden")(hidden)
    print(hidden.shape)
    print(sport_hidden.shape)
    print(dport_hidden.shape)
    print(proto_hidden.shape)
    sport_output = tf.keras.layers.Softmax(1, name="sport_output")(sport_hidden)
    dport_output = tf.keras.layers.Softmax(1, name="dport_output")(dport_hidden)
    proto_output = tf.keras.layers.Softmax(1, name="proto_output")(proto_hidden)
    cont_output = tf.keras.layers.Dense(num_features-3, name="cont_output")(hidden)

    model = tf.keras.Model(inputs=input, outputs=[sport_output, dport_output, proto_output, cont_output],
                           name="Generator")
    opt = tf.keras.optimizers.RMSprop(learning_rate=0.00005)
    # model.compile(loss=wasserstein_loss, optimizer=opt)

    return model

# Assuming input_dim is 100 for example
input_dim = 100

# Create a random input tensor for testing
random_input = np.random.rand(1, input_dim)

# Instantiate the generator model
generator_model = make_generator_model(input_dim)

# Forward pass through the generator
output_tensors = generator_model.predict(random_input)

# Display the shapes after each step
print("Random Input Shape:", random_input.shape)
print("Hidden Layer Output Shape:", output_tensors[0].shape)
print("Sport Output Shape:", output_tensors[1].shape)
print("Dport Output Shape:", output_tensors[2].shape)
print("Proto Output Shape:", output_tensors[3].shape)
print("Continuous Output Shape:", output_tensors[4].shape)


(None, 80)
(None, 80)
(None, 23030)
(None, 14372)
(None, 7)
1/1 [==============================] - 0s 90ms/step
Random Input Shape: (1, 100)
Hidden Layer Output Shape: (1, 23030)
Sport Output Shape: (1, 14372)
Dport Output Shape: (1, 7)
Proto Output Shape: (1, 27)


IndexError: list index out of range